In [1]:
!pip install pyathena
from athena_querying import AthenaQuery
from athena_common_queries import *

    100% |████████████████████████████████| 51kB 14.4MB/s ta 0:00:01
    100% |████████████████████████████████| 829kB 21.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
You are using pip version 10.0.1, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
    
    
try:
    import dateparser
except:
    print("Failed ot import dateparser, trying to install it")
    !{sys.executable} -m pip install dateparser
    import dateparser
    print("Successfully installed")
    
import data_pier_querying

Failed ot import dateparser, trying to install it


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


    100% |████████████████████████████████| 358kB 16.9MB/s ta 0:00:01
    100% |████████████████████████████████| 696kB 20.0MB/s ta 0:00:01
You are using pip version 10.0.1, however version 20.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [4]:
from datetime import datetime, timedelta, date

# Settings

In [5]:
num_days_to_query = 5
#from_datetime = datetime.now() - timedelta(days = 5)
from_datetime = datetime(year=2020, month=1, day=10)
to_datetime = from_datetime+ timedelta(days=num_days_to_query)

# Athena (Kinesis) Data

In [6]:
aq = AthenaQuery()

In [7]:
aq.connect()

In [8]:
athena_database = "ms_data_lake_production"
athena_raw_events_table = "ms_data_stream_production_processed"

In [9]:
#query = "select context.page_url, body.event_name, count(*) from "+athena_database+"."+athena_raw_events_table
#query += " where partition_0='2019' and partition_1>='12' and partition_2>='05' group by 1,2"

In [10]:
# I've removed the device_type data to save memory, but it would be useful.
query = create_generic_event_query(from_datetime, to_datetime, include_device_type_data=False, interpret_urls=False)

full_query = "select * from (%s) where country_code ='sg'" %query

In [11]:
print(full_query)

select * from (
    
    SELECT 
          CAST("from_iso8601_timestamp"("sent_at") AS timestamp) "sent_at_timestamp"
    , "sent_at"
    , "type"
    , "body"."event_name"
    , "body"."data"."status"
    , "user"."anonymous_id"
    , "user"."amp_id"
    , "context"."page_url"
    , "context"."referrer"
 
    
    
    FROM
      ms_data_lake_production.ms_data_stream_production_processed
    
    
    WHERE true -- makes query composition easier
    
 AND 
  (
 partition_0 >= '2020'
 AND partition_1 >= '01'
 AND partition_2 >= '10'
 OR (
 partition_0 >= '2020'
 AND partition_1 > '01'
 ) 
 OR (
 partition_0 > '2020'
 ) 
)
 AND ((partition_0 <= '2020'
	 AND partition_1 <= '01'
	 AND partition_2 <= '15'
) 
 OR (
	 partition_0 <= '2020'
	 AND partition_1 < '01'
) 
 OR (
	 partition_0 < '2020'
) 
)
 AND CAST(from_iso8601_timestamp(sent_at) AS timestamp)  between CAST(from_iso8601_timestamp('2020-01-10T00:00:00') AS timestamp) AND CAST(from_iso8601_timestamp('2020-01-15T00:00:00') AS times

In [12]:
athena_full_events_df = aq.query(query)

In [13]:
# Set types to speed queries and save on memory
athena_full_events_df = athena_full_events_df.astype({ "type":"category"
    , "event_name":"category"
    , "status":"category"}, copy=False)

In [14]:
athena_full_events_df.dtypes

sent_at_timestamp      object
sent_at                object
type                 category
event_name           category
status               category
anonymous_id           object
amp_id                 object
page_url               object
referrer               object
dtype: object

In [15]:
athena_full_events_df.head(5)

,sent_at_timestamp,sent_at,type,event_name,status,anonymous_id,amp_id,page_url,referrer
0,2020-01-13 19:13:38.654,2020-01-13T19:13:38.654Z,page,PageView,NaN,cc160631-0b7c-4191-b29b-58a32b60bcd0,NaN,https://www.moneysmart.sg/embed/52174e7b8d839f...,https://s0.2mdn.net/dfp/509788/70424308/157292...
1,2020-01-13 00:00:00.596,2020-01-13T00:00:00.596000+00:00,page,PageView,NaN,NaN,amp-vXPuO6NvZYXNPpZ8unTwBg,https://www.moneysmart.id/cara-menghasilkan-ua...,NaN
2,2020-01-13 19:13:37.397,2020-01-13T19:13:37.397Z,page,PageView,NaN,cc160631-0b7c-4191-b29b-58a32b60bcd0,NaN,https://blog.moneysmart.sg/transportation/chea...,https://www.google.com.sg/
3,2020-01-13 19:13:39.508,2020-01-13T19:13:39.508Z,event,UserView.WidgetLoad,NaN,cc160631-0b7c-4191-b29b-58a32b60bcd0,NaN,https://www.moneysmart.sg/embed/52174e7b8d839f...,https://s0.2mdn.net/dfp/509788/70424308/157292...
4,2020-01-13 19:13:42.230,2020-01-13T19:13:42.230Z,event,Reading,Article Body 50,21b240ed-e153-4d91-8a8a-56b4e1e26f12,NaN,https://blog.moneysmart.sg/career/highest-payi...,https://www.google.com/


# Segment Data

NB: screwed up, and can use the tracks table, rather than individual event tables, so a lot of this is pointless.

In [16]:
#from importlib import reload
#reload(data_pier_querying)

In [17]:
dp_querying = data_pier_querying.DataPierQuerying()
dp_querying.connect()

In [18]:
tables_df = dp_querying.query_to_dataframe("select * from information_schema.tables")

In [19]:
segment_event_tables_df = tables_df[tables_df.table_schema=="moneysmartsg_prod"]["table_name"]


In [20]:
# These are taken from the dictionary in https://docs.google.com/spreadsheets/d/1HICh77BoGMIat9K3NPwz3pBayJWiAr0ohAlTuv7dr80/edit#gid=1882048411
#but actually it turns out there should be more than this, and don't need to do it this way.
expected_events_str = """
LeadGeneration.ClickConversion
LeadGeneration.FormStepCompleted
LeadGeneration.FormSubmitted
LeadGeneration.PaymentCompleted
LeadGeneration.ThankYou
LeadGeneration.RedirectCompleted
UserEngagement.ShowedMoreDetails
UserEngagement.ViewedMoreDetails
UserEngagement.SortedList
UserEngagement.UsedHelpHints
UserEngagement.ClickedMenuItem
UserEngagement.QuestionAnswered
UserEngagement.ShowMoreFilter
UserEngagement.ShowMoreOptions
UserEngagement.ClickedFilter
UserEngagement.ButtonClick
UserAuth.LoggedIn
UserAuth.RegisteredAccount
UserAuth.LoggedOut
UserFeedback.ModalDisplayed
UserFeedback.MoodSubmitted
UserFeedback.FeedbackSubmitted
UserFeedback.MoreFeedback
ABTest.Conversion
UserView.WidgetLoad
EmailCapture
PageView
Sharing
Reading
NewsLetterPopup
"""
expected_events = [z.strip() for z in expected_events_str.split("\n") if len(z.strip())>0]

In [21]:


expected_events_and_segment_tables = []
special_maps = {
    "PageView": "pages"
}
for event in expected_events:
    if event in special_maps:
        new_event_name = special_maps[event]
    else:
        new_event_name = ""
        for i, c in enumerate(event):
            if i==0:new_event_name+=c.lower()
            elif str.isupper(c): 
                if i>0 and event[i-1]!=".":
                    new_event_name += "_"
                new_event_name += c.lower()
            elif c==".": new_event_name += "_"
            else: new_event_name+= c
    expected_events_and_segment_tables.append([event, new_event_name])

In [22]:
expected_events_and_segment_tables

[['LeadGeneration.ClickConversion', 'lead_generation_click_conversion'],
 ['LeadGeneration.FormStepCompleted', 'lead_generation_form_step_completed'],
 ['LeadGeneration.FormSubmitted', 'lead_generation_form_submitted'],
 ['LeadGeneration.PaymentCompleted', 'lead_generation_payment_completed'],
 ['LeadGeneration.ThankYou', 'lead_generation_thank_you'],
 ['LeadGeneration.RedirectCompleted', 'lead_generation_redirect_completed'],
 ['UserEngagement.ShowedMoreDetails', 'user_engagement_showed_more_details'],
 ['UserEngagement.ViewedMoreDetails', 'user_engagement_viewed_more_details'],
 ['UserEngagement.SortedList', 'user_engagement_sorted_list'],
 ['UserEngagement.UsedHelpHints', 'user_engagement_used_help_hints'],
 ['UserEngagement.ClickedMenuItem', 'user_engagement_clicked_menu_item'],
 ['UserEngagement.QuestionAnswered', 'user_engagement_question_answered'],
 ['UserEngagement.ShowMoreFilter', 'user_engagement_show_more_filter'],
 ['UserEngagement.ShowMoreOptions', 'user_engagement_show_m

## Check for missing tables

Expect some random events not to be in Segment, or blog specific ones that haven't been deployed to SG and HK

In [23]:
# Check all the event tables exist
expected_event_segment_tables = [z[1] for z in expected_events_and_segment_tables]
segment_table_names = segment_event_tables_df.to_list()
missing_event_tables = [z for z in expected_event_segment_tables if z not in segment_table_names]
missing_event_tables

['user_engagement_used_help_hints',
 'user_engagement_clicked_menu_item',
 'user_feedback_modal_displayed',
 'user_feedback_more_feedback',
 'a_b_test_conversion',
 'sharing',
 'news_letter_popup']

In [24]:
expected_events_and_segment_tables

[['LeadGeneration.ClickConversion', 'lead_generation_click_conversion'],
 ['LeadGeneration.FormStepCompleted', 'lead_generation_form_step_completed'],
 ['LeadGeneration.FormSubmitted', 'lead_generation_form_submitted'],
 ['LeadGeneration.PaymentCompleted', 'lead_generation_payment_completed'],
 ['LeadGeneration.ThankYou', 'lead_generation_thank_you'],
 ['LeadGeneration.RedirectCompleted', 'lead_generation_redirect_completed'],
 ['UserEngagement.ShowedMoreDetails', 'user_engagement_showed_more_details'],
 ['UserEngagement.ViewedMoreDetails', 'user_engagement_viewed_more_details'],
 ['UserEngagement.SortedList', 'user_engagement_sorted_list'],
 ['UserEngagement.UsedHelpHints', 'user_engagement_used_help_hints'],
 ['UserEngagement.ClickedMenuItem', 'user_engagement_clicked_menu_item'],
 ['UserEngagement.QuestionAnswered', 'user_engagement_question_answered'],
 ['UserEngagement.ShowMoreFilter', 'user_engagement_show_more_filter'],
 ['UserEngagement.ShowMoreOptions', 'user_engagement_show_m

In [25]:
# Removing the missing ones from the query list
events_and_tables_to_get_from_data_pier = [z for z in expected_events_and_segment_tables if z[1] not in missing_event_tables]

# Removing a problematic one (doesn't have context_page_url in it, and very unimportant
events_and_tables_to_get_from_data_pier = [z for z in events_and_tables_to_get_from_data_pier if z[1] not in ["user_auth_logged_out",]]

In [26]:
len(events_and_tables_to_get_from_data_pier)

22

In [27]:
cols = dp_querying.query_to_dataframe("""
select column_name, data_type, count(*) from information_schema.columns 
where 
table_name in  ('"""+"','".join([z[1] for z in events_and_tables_to_get_from_data_pier])+"""')
and table_schema='moneysmartsg_prod'

group by 1,2
""")

In [28]:
#cols.sort_values(["count"])

In [29]:
segment_columns_to_query = [
    # "sent_at", - don't use this, use timestamp
    "timestamp",
    #"event", - going to get that implied from the table.
    # "status", # TODO: would like to have this, but not sure which column, or which tables.  Maybe just not used much, so only do for the 4 tables.
    "anonymous_id",
    "context_page_url",
    # "referrer", #maybe only used in pages table??
    "context_ip", 
    "context_user_agent"]

In [30]:
segment_date_constraint = " timestamp >= '%s' and timestamp < '%s' " % (from_datetime.isoformat(), to_datetime.isoformat())

In [31]:
query = "select {cols} from moneysmartsg_prod.{table} where {date_constraint}".format(cols=", ".join(segment_columns_to_query), 
                                                                   table="pages",
                                                                   date_constraint =segment_date_constraint)

In [32]:
query

"select timestamp, anonymous_id, context_page_url, context_ip, context_user_agent from moneysmartsg_prod.pages where  timestamp >= '2020-01-10T00:00:00' and timestamp < '2020-01-15T00:00:00' "

In [33]:
pages = dp_querying.query_to_dataframe("select * from  moneysmartsg_prod.pages where  timestamp > '2020-01-08T15:24:00.263926' and timestamp < '2020-01-15T15:24:00.263966' limit 1000")

In [34]:
click_conversions = dp_querying.query_to_dataframe("select * from  moneysmartsg_prod.lead_generation_click_conversion where  timestamp > '2020-01-08T15:24:00.263926' and timestamp < '2020-01-15T15:24:00.263966' limit 1000")

In [35]:
click_conversions.head()

,id,received_at,channel,context_ip,context_library_name,context_library_version,context_page_referrer,context_user_agent,event_text,product_name,...,product_category_attributes_meta_description,product_category_attributes_meta_title,product_category_type,product_category_attributes_name,product_category_attributes_short_description,product_category_attributes_masthead_title,product_category_attributes_description,product_category_attributes_default,product_category_attributes_slug,product_category_attributes_position
0,ajs-28857690310f6448f49069f94d4e48ec,2020-01-09 02:30:19.509000+00:00,car-insurance,54.255.226.249,analytics.js,3.10.1,None,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,LeadGeneration.ClickConversion,Classic,...,None,None,None,None,None,None,None,None,None,None
1,ajs-319c118f2d0762d259ba66fa47f018e9,2020-01-08 15:33:04.210000+00:00,car-insurance,116.87.146.242,analytics.js,3.10.1,https://www.moneysmart.sg/car-insurance/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,LeadGeneration.ClickConversion,FWD Classic,...,None,None,None,None,None,None,None,None,None,None
2,ajs-59d8a1f8e28e7c2838d5c199635d7264,2020-01-08 15:26:45.031000+00:00,None,119.74.42.156,analytics.js,3.10.1,https://search.yahoo.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,LeadGeneration.ClickConversion,Standard Chartered Unlimited Cashback Credit Card,...,None,None,None,None,None,None,None,None,None,None
3,ajs-6ff3941406c7081456e7c0c2b2d1d089,2020-01-08 15:48:38.556000+00:00,None,27.125.181.147,analytics.js,3.10.1,https://www.google.co.jp/,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,LeadGeneration.ClickConversion,DBS Live Fresh Card,...,None,None,None,None,None,None,None,None,None,None
4,ajs-98805c85d50889f85d0f016873bd68d5,2020-01-08 15:58:22.972000+00:00,None,138.75.26.184,analytics.js,3.10.1,https://www.moneysmart.sg/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,LeadGeneration.ClickConversion,ICBC Global Travel MasterCard Credit Card,...,None,None,None,None,None,None,None,None,None,None


In [36]:
d = dp_querying.query_to_dataframe("select * from moneysmartsg_prod.tracks limit 10")

In [37]:
for a in d.columns: print(a)

id
received_at
context_page_url
original_timestamp
context_library_name
context_library_version
context_page_referrer
context_campaign_source
context_ip
user_id
event_text
uuid_ts
context_campaign_content
context_user_agent
event
context_page_path
context_page_search
context_page_title
sent_at
timestamp
anonymous_id
context_campaign_medium
context_campaign_name
context_campaign_term
context_locale


In [38]:
d["event_text"]

0                             Reading
1                             Reading
2                             Reading
3    LeadGeneration.ShowedMoreDetails
4                             Reading
5                             Reading
6                 UserView.WidgetLoad
7                             Reading
8                             Reading
9                             Reading
Name: event_text, dtype: object

In [39]:

for a in click_conversions.keys():
    print(a)

id
received_at
channel
context_ip
context_library_name
context_library_version
context_page_referrer
context_user_agent
event_text
product_name
provider
context_page_title
event_type
is_paid
list_position
product_id
uuid_ts
context_page_path
context_page_url
provider_id
anonymous_id
auth_status
event
original_timestamp
sent_at
timestamp
action
context_page_search
is_recommended
is_sponsored
source_id
source
affiliate_page_type
affiliate_location
affiliate_category
context_campaign_name
context_campaign_medium
context_campaign_source
context_campaign_content
form_name
page_path
widget_type
affliate_widget_type
page_referrer
provider_slug
product_category_slug
country
page_type
product_slug
product
product_category
context_campaign_term
context_locale
product_category_attributes_meta_description
product_category_attributes_meta_title
product_category_type
product_category_attributes_name
product_category_attributes_short_description
product_category_attributes_masthead_title
product_cate

In [40]:
dp_querying.query_to_dataframe("""SELECT
    nmsp_parent.nspname AS parent_schema,
    parent.relname      AS parent,
    nmsp_child.nspname  AS child_schema,
    child.relname       AS child
FROM pg_inherits
    JOIN pg_class parent            ON pg_inherits.inhparent = parent.oid
    JOIN pg_class child             ON pg_inherits.inhrelid   = child.oid
    JOIN pg_namespace nmsp_parent   ON nmsp_parent.oid  = parent.relnamespace
    JOIN pg_namespace nmsp_child    ON nmsp_child.oid   = child.relnamespace
WHERE parent.relname='%s';""")%"pages"

,parent_schema,parent,child_schema,child


In [41]:
pd.get_option("display.max_colwidth", 200)
indexes = dp_querying.query_to_dataframe("""SELECT
    indexname,
    indexdef
FROM
    pg_indexes
WHERE
    tablename = '%s';""" % "pages")

for a in indexes.values:
    print(a)

['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmarthk_prod.pages USING btree (id)']
['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmartsg_prod.pages USING btree (id)']
['pages_timestamp_idx'
 'CREATE INDEX pages_timestamp_idx ON moneysmartsg_prod.pages USING btree ("timestamp")']
['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmarthk_dev.pages USING btree (id)']
['pages_pkey'
 'CREATE UNIQUE INDEX pages_pkey ON moneysmartsg_dev.pages USING btree (id)']


In [ ]:
query_segment_by_table = False #really shouldn't set this to true, didn't realise correct method.  Also need to add country stuff

segment_schemas = ["moneysmartsg_prod", "moneysmarthk_prod"]
# The meat of it
start_time = datetime.now()
event_name_to_rows = {}
if query_segment_by_table:
    for country_schema in segment_schemas:
        for i, (event_name, table_name) in enumerate(events_and_tables_to_get_from_data_pier):
            table_start_time = datetime.now()
            print("querying table %s / %s (%i/%i)" % (table_name, event_name, i+1, len(events_and_tables_to_get_from_data_pier)))
            query = "select {cols} from {schema}.{table} where {date_constraint}".format(cols=", ".join(segment_columns_to_query), 
                                                                           table=table_name,
                                                                           date_constraint =segment_date_constraint, schema=country_schema)

            events = dp_querying.query_to_dataframe(query)
            events["event_name"] = event_name #fills the entire column with the same value
            print("Got %i events"% len(events))
            event_name_to_rows[event_name]=events

            table_download_time = (datetime.now()-table_start_time).total_seconds()
            time_since_start = (datetime.now()-start_time).total_seconds()
            print("It took %.1f seconds to download from the table (%.1f seconds overall)" %(table_download_time, time_since_start))
            print()
            # if i>4:break


        # Merge tables
        segment_combined_df = pd.DataFrame()
        #combined_df = pd.DataFrame(columns=event_name_to_rows["LeadGeneration.ClickConversion"].columns)
        """for event_name, event_df in event_name_to_rows.items():
            print(len(event_df))
            combined_df.append(event_df, ignore_index=True)
            print(len(combined_df))
        #combined_df.astype({"event_name":"category"})
        """

        segment_combined_df = combined_df.append(list(event_name_to_rows.values()))
    
    
else:
    segment_columns_to_query_full = segment_columns_to_query + ["event_text",]
    tables_to_query = ["pages", "tracks"]
    all_event_dfs = []
    segment_combined_df = pd.DataFrame()
    for country_schema in segment_schemas:
        for table_name in tables_to_query:
            table_start_time = datetime.now()
            if table_name!="pages":
                cols_to_query = segment_columns_to_query_full
            else:
                cols_to_query = segment_columns_to_query
            print("querying table %s.%s" % (country_schema, table_name))
            print(cols_to_query)
            query = "select {cols} from {schema}.{table} where {date_constraint}".format(cols=", ".join(cols_to_query), 
                                                                           table=table_name,
                                                                           date_constraint =segment_date_constraint, schema=country_schema)

            events = dp_querying.query_to_dataframe(query)
            
            print("Got %i events"% len(events))
            #all_event_dfs.append(events)
            
            if table_name =="pages":
                events["event_text"] = "PageView" # fills the whole column
            table_download_time = (datetime.now()-table_start_time).total_seconds()
            time_since_start = (datetime.now()-start_time).total_seconds()
            print("merging")
            segment_combined_df = segment_combined_df.append(events)
            print("It took %.1f seconds to download from the table (%.1f seconds overall)" %(table_download_time, time_since_start))
            print()
            
        

querying table moneysmartsg_prod.pages
['timestamp', 'anonymous_id', 'context_page_url', 'context_ip', 'context_user_agent']
Got 806166 events
merging
It took 8.6 seconds to download from the table (8.6 seconds overall)

querying table moneysmartsg_prod.tracks
['timestamp', 'anonymous_id', 'context_page_url', 'context_ip', 'context_user_agent', 'event_text']


In [ ]:
if not query_segment_by_table:
    segment_combined_df.rename(columns={"event_text":"event_name"}, inplace=True)

In [ ]:
len(all_event_dfs)

In [ ]:
segment_combined_df.rename(columns={"context_page_url":"page_url"}, inplace=True)
segment_combined_df.head(5)

# Merging Segment and Kinesis Events

In [ ]:
# Make names clear e.g. s_...

# Check the timezone / timestamps match
# Athena raw stuff is in UTC, not SG time.  So 2020-01-19T00:04:04.443Z is 8:05am Singapore time.
# whereas Segment is stored with tiemzone at UTC.  So, could convert them all.
# TODO: But it does meant that there's a lot of events coming at the day boundary.

In [ ]:
athena_full_events_df.head(2)

In [ ]:
athena_full_events_df.dtypes

In [ ]:
segment_combined_df.dtypes

In [ ]:
# Group by columns to get around date inaccuracy issue
cols_to_group_by = ["anonymous_id", "event_name", "page_url", "date"] #, "context_ip", "context_user_agent"] #TODO: add IP address

print("Grouping by %s"% ", ".join(cols_to_group_by))

print("Fixing dates before grouping")
print("... for Segment")
segment_combined_df["date"] = segment_combined_df.apply(lambda row: row.timestamp.date().isoformat(), axis=1) # making this a string
print("... for athena")
athena_full_events_df["date"] = athena_full_events_df.apply(lambda row: row.sent_at[:10], axis=1)
# super-slow,so moving to using strings athena_full_events_df["date"] = athena_full_events_df.apply(lambda row: dateparser.parse(row.sent_at_timestamp).date(), axis=1)  #conversion from string might not be needed in the future; using dateparser as more robust, also slow

#going to reduce the number of columns to make it safer, then can go back and look for user agents etc (can do a mapping of anonymous_id to user_agent for instance.)




In [ ]:
print("Setting sensible data types for the columns to group by")
data_type_mappings = {"event_name":"category", "date":"category"}
segment_combined_df.astype(data_type_mappings, copy=False)
athena_full_events_df.astype(data_type_mappings, copy=False)

Setting sensible data types for the columns to group by


,sent_at_timestamp,sent_at,type,event_name,status,anonymous_id,amp_id,page_url,referrer,date
0,2020-01-13 19:13:38.654,2020-01-13T19:13:38.654Z,page,PageView,NaN,cc160631-0b7c-4191-b29b-58a32b60bcd0,NaN,https://www.moneysmart.sg/embed/52174e7b8d839f...,https://s0.2mdn.net/dfp/509788/70424308/157292...,2020-01-13
1,2020-01-13 00:00:00.596,2020-01-13T00:00:00.596000+00:00,page,PageView,NaN,NaN,amp-vXPuO6NvZYXNPpZ8unTwBg,https://www.moneysmart.id/cara-menghasilkan-ua...,NaN,2020-01-13
2,2020-01-13 19:13:37.397,2020-01-13T19:13:37.397Z,page,PageView,NaN,cc160631-0b7c-4191-b29b-58a32b60bcd0,NaN,https://blog.moneysmart.sg/transportation/chea...,https://www.google.com.sg/,2020-01-13
3,2020-01-13 19:13:39.508,2020-01-13T19:13:39.508Z,event,UserView.WidgetLoad,NaN,cc160631-0b7c-4191-b29b-58a32b60bcd0,NaN,https://www.moneysmart.sg/embed/52174e7b8d839f...,https://s0.2mdn.net/dfp/509788/70424308/157292...,2020-01-13
4,2020-01-13 19:13:42.230,2020-01-13T19:13:42.230Z,event,Reading,Article Body 50,21b240ed-e153-4d91-8a8a-56b4e1e26f12,NaN,https://blog.moneysmart.sg/career/highest-payi...,https://www.google.com/,2020-01-13
5,2020-01-13 19:13:40.312,2020-01-13T19:13:40.312Z,event,Reading,Article Body 25,1da45cef-e1bd-434f-9814-bcb972cff6d1,NaN,https://www.moneysmart.tw/articles/%E5%BB%89%E...,https://www.google.com/,2020-01-13
6,2020-01-13 19:13:40.466,2020-01-13T19:13:40.466Z,event,LeadGeneration.ClickedApplyButton,NaN,87562ee3-294d-43cf-86f2-e77568315ec1,NaN,https://www.moneysmart.hk/zh-hk/credit-cards/a...,http://m.facebook.com,2020-01-13
7,2020-01-13 19:13:40.291,2020-01-13T19:13:40.291Z,event,Reading,Article Body 75,9d254142-d35a-4721-b53d-8c96ad6a2c7c,NaN,https://blog.moneysmart.sg/property/landed-pro...,https://www.google.com/,2020-01-13
8,2020-01-13 19:14:11.033,2020-01-13T19:14:11.033Z,page,PageView,NaN,c75c1803-5a7a-49c8-8778-dc98a8897016,NaN,https://www.moneysmart.hk/zh-hk/credit-cards/b...,https://www.moneysmart.hk/zh-hk/credit-cards,2020-01-13
9,2020-01-13 19:13:41.884,2020-01-13T19:13:41.884Z,event,LeadGeneration.Conversion,NaN,87562ee3-294d-43cf-86f2-e77568315ec1,NaN,https://www.moneysmart.hk/zh-hk/credit-cards/s...,https://www.moneysmart.hk/zh-hk/credit-cards/a...,2020-01-13


In [ ]:
segment_combined_df.head()[cols_to_group_by]

,anonymous_id,event_name,page_url,date
0,b843bc94-64dc-4dca-8978-08877ddaf7eb,PageView,https://blog.moneysmart.sg/travel/singapore-pu...,2020-01-10
1,b843bc94-64dc-4dca-8978-08877ddaf7eb,PageView,https://www.moneysmart.sg/embed/7432e8a28bd976...,2020-01-10
2,0263aa86-16ca-46d3-b7ee-37bb7665f1da,PageView,https://blog.moneysmart.sg/transportation/erp-...,2020-01-10
3,0263aa86-16ca-46d3-b7ee-37bb7665f1da,PageView,https://www.moneysmart.sg/embed/52174e7b8d839f...,2020-01-10
4,c25624f4-6ccf-4add-8363-226926ca9737,PageView,https://blog.moneysmart.sg/fixed-deposits/best...,2020-01-10


In [ ]:
athena_full_events_df.head()[cols_to_group_by]

,anonymous_id,event_name,page_url,date
0,cc160631-0b7c-4191-b29b-58a32b60bcd0,PageView,https://www.moneysmart.sg/embed/52174e7b8d839f...,2020-01-13
1,NaN,PageView,https://www.moneysmart.id/cara-menghasilkan-ua...,2020-01-13
2,cc160631-0b7c-4191-b29b-58a32b60bcd0,PageView,https://blog.moneysmart.sg/transportation/chea...,2020-01-13
3,cc160631-0b7c-4191-b29b-58a32b60bcd0,UserView.WidgetLoad,https://www.moneysmart.sg/embed/52174e7b8d839f...,2020-01-13
4,21b240ed-e153-4d91-8a8a-56b4e1e26f12,Reading,https://blog.moneysmart.sg/career/highest-payi...,2020-01-13


In [ ]:
# athena_full_events_df timestamp

print("Grouping by %s"%cols_to_group_by)
segment_grouped_df = segment_combined_df.groupby(cols_to_group_by).size().reset_index(name='s_count') #size preserves nulls, this sets the column to s_count

athena_grouped_df = athena_full_events_df.groupby(cols_to_group_by).size().reset_index(name='k_count')

# segment_combined_df.rename(columns = {"context_ip":"s_context_ip", "context_user_agent":"s_context_user_agent"}) 

Grouping by ['anonymous_id', 'event_name', 'page_url', 'date']


In [ ]:
athena_grouped_df.head()

,anonymous_id,event_name,page_url,date,k_count
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,2
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-14,1
2,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog3.moneysmart.sg/career/5-easy-side...,2020-01-13,1
3,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1
4,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,3


In [ ]:
# Actually join them

# set the column count names

merged_df = segment_grouped_df.merge(athena_grouped_df, how='outer', on=cols_to_group_by )

#Fill in the empty counts with 0s

merged_df["s_count"].fillna(0, inplace=True)
merged_df["k_count"].fillna(0, inplace=True)

In [ ]:
merged_df

,anonymous_id,event_name,page_url,date,s_count,k_count
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,2.0,2.0
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-13,1.0,0.0
2,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-14,1.0,1.0
3,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1.0,1.0
4,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,3.0,3.0
5,00000b54-600a-4de2-8700-fd9885252dca,UserView.WidgetLoad,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1.0,1.0
6,000034a2-e973-4108-b920-0681877d4fc0,PageView,https://blog.moneysmart.sg/budgeting/cheapest-...,2020-01-10,1.0,1.0
7,00006910-c807-47f9-b57e-0b5e50f91ce1,PageView,https://blog.moneysmart.hk/zh-hk/mortgage/%E6%...,2020-01-13,1.0,1.0
8,00006910-c807-47f9-b57e-0b5e50f91ce1,Reading,https://blog.moneysmart.hk/zh-hk/mortgage/%E6%...,2020-01-13,3.0,3.0
9,0000746e-0fc2-4d90-8962-e9d9553419ed,PageView,https://blog.moneysmart.hk/zh-hk/budgeting/%E8...,2020-01-13,1.0,1.0


In [ ]:
merged_df.groupby(["event_name"]).sum()

,s_count,k_count
event_name,,
Display user feedback form,3.0,0.0
EmailCapture,839.0,848.0
LeadGeneration.ClickConversion,1656.0,1889.0
LeadGeneration.ClickedApplyButton,5504.0,5825.0
LeadGeneration.ClickedCTA,6111.0,5429.0
LeadGeneration.ClickedGoToSite,446.0,0.0
LeadGeneration.Conversion,13757.0,12736.0
LeadGeneration.FormStepCompleted,1707.0,1722.0
LeadGeneration.FormSubmitted,8144.0,8306.0


# Add Filtering Metadata

* is url blog / shop / ...
* country

In [ ]:
from urllib.parse import urlparse, parse_qs

In [155]:
def get_metadata_from_url(url):
    p = urlparse(url.lower())
    
    #urlparse("https://www-new.moneysmart.sg/rabbit/mouse/?a=b")
    #ParseResult(scheme='https', netloc='www-new.moneysmart.sg', path='/rabbit/mouse/', params='', query='a=b', fragment='')
    
    
    nl = p.netloc
    
    page_type = ""
    stripped_path = p.path.strip("/")
    
    #blog (for SG and HK)
    if "moneysmart.tw" in nl or "moneysmart.ph" in nl or "moneysmart.id" in nl or 'blog.moneysmart' in nl or 'blog-new' in nl or 'blog3' in nl:
        page_type = "blog"
    
    #LPS
    elif stripped_path.endswith("ms"):
        page_type = "lps"
    
    #interstitial
    elif "iss.moneysmart" in nl or stripped_path.endswith("apply") or stripped_path.endswith("redirect"):
        page_type = "iss"
        
    #unbounce
    elif "get.moneysmart" in nl:
        page_type = "unbounce"
        
    #embed     , "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 5) like '/embed/%' as is_embed
    
    
    #else shop
    else:
        page_type = "shop"
        
        
        
    
    #ab test side , CAST("strpos"("context"."page_url", '://www-new.') AS boolean) OR CAST("strpos"("context"."page_url", '://www3.') AS boolean)  OR CAST("strpos"("context"."page_url", '://blog3.') AS boolean) as "is_test"
    if "www-new." in nl or "www3." in nl or "blog3." in nl:
        ab_test = "test"
    else:
        ab_test = "control"
    
    slug = "/"+stripped_path
    
    if slug.startswith("/en/") or slug.startswith("/zh-hk/") or slug=="/en" or slug=="/zh-hk":
        slug_root = "/"+stripped_path.split("/")[1]
    else:
        slug_root = "/"+stripped_path.split("/")[0]
    
    
    
    
    """
     , CASE WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.sg%' THEN 'sg' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.hk%' THEN 'hk' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.id%' THEN 'id' 
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.ph%' THEN 'ph'
        WHEN "regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) LIKE '%moneysmart.tw%' THEN 'tw' 
        ELSE null END as country_code
    """
    
    country_code = ""
    if "moneysmart.sg" in nl:
        country_code = "sg"
    elif "moneysmart.hk" in nl:
        country_code = "hk"
    elif "moneysmart.id" in nl:
        country_code = "id"
    elif "moneysmart.ph" in nl:
        country_code = "ph"
    elif "moneysmart.tw" in nl:
        country_code = "tw"
    elif "moneysmart.com" in nl:
        country_code = "ww" #worldwide
    else:
        country_code = "??"
    
    
    #return {"page_type":page_type, "path":path, "ab_test":ab_test, "country_code":country_code}
    return [page_type, slug, slug_root, ab_test, country_code]
    
    



In [117]:
get_metadata_from_url("https://www-new.moneysmart.sg/rabbit/headlight/?scary=True")

['shop', '/rabbit/headlight', '/rabbit', 'test', 'sg']

In [118]:
get_metadata_from_url("https://blog.moneysmart.ph/rabbit/headlight/?scary=True")

['blog', '/rabbit/headlight', '/rabbit', 'control', 'ph']

In [119]:
get_metadata_from_url("https://blog3.moneysmart.tw")

['blog', '/', '/', 'test', 'tw']

In [120]:
get_metadata_from_url("https://www.moneysmart.hk/zh-hk/credit-cards/")

['shop', '/zh-hk/credit-cards', '/credit-cards', 'control', 'hk']

In [121]:
#This is a bit slow
metadata_df = merged_df.apply(lambda x: pd.Series(get_metadata_from_url(x.page_url)), axis=1)#, index=["page_type", "path", "ab_test", "country_code"])

In [122]:
metadata_df.rename(columns={0:"page_type", 1:"slug", 2:"slug_root", 3:"ab_test", 4:"country_code"}, inplace=True)

In [123]:
metadata_df.head()

,page_type,slug,slug_root,ab_test,country_code
0,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
2,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
3,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg
4,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg


In [124]:
merged_df_with_meta = pd.concat([merged_df, metadata_df], axis=1)

In [125]:
# Set some sensible data types to speed it all up
#merged_df_with_meta.astype({"page_type":"category", "slug":"category"})
merged_df_with_meta = merged_df_with_meta.astype({"page_type":"category", "slug":"category", "ab_test":"category", "country_code":"category", "s_count":"int", "k_count":"int"})

In [126]:
merged_df_with_meta.head()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,2,2,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-13,1,0,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
2,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-14,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
3,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg
4,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,3,3,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg


In [127]:
merged_df_with_meta[(merged_df_with_meta.s_count>1) & (merged_df_with_meta.k_count>1)].head()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,2,2,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
4,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,3,3,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
8,00006910-c807-47f9-b57e-0b5e50f91ce1,Reading,https://blog.moneysmart.hk/zh-hk/mortgage/%E6%...,2020-01-13,3,3,blog,/zh-hk/mortgage/%e6%8c%89%e6%8f%ad%e4%bf%9d%e9...,/mortgage,control,hk
10,0000746e-0fc2-4d90-8962-e9d9553419ed,Reading,https://blog.moneysmart.hk/zh-hk/budgeting/%E8...,2020-01-13,2,2,blog,/zh-hk/budgeting/%e8%a3%9d%e4%bf%ae%e4%bd%ac-%...,/budgeting,control,hk
13,00008e20-54bd-495c-82ce-ebc6193bb1c9,Reading,https://blog.moneysmart.sg/dining/starbucks-si...,2020-01-10,3,3,blog,/dining/starbucks-singapore-menu-prices,/dining,control,sg


# Top Level Checks

In [128]:
def group_by_and_show_count_difference(df, group_by_cols, with_styling=True):
    """
    This expects counts in s_count and k_count
    """
    
    grouped = df.groupby(group_by_cols).sum().reset_index()
    
    grouped["k_vs_s_%"] = grouped.apply(lambda row:(999 if row.k_count else 0) if row.s_count==0 else round(((row.k_count - row.s_count)/row.s_count)*100, 1), axis=1 )
    grouped = grouped[(grouped.k_count>0) | (grouped.s_count>0)] # filters out NaNs after grouping
    
    

    return grouped

In [129]:
def colour_grouped_table(df):

        

    def color_how_good(value):
        if isinstance(value, str):
            return
        av = abs(value)
        if av<2:
            c = "green"
        elif value <0:
            c =  "red"

        else:
            c= "blue"

        if av>20:
            return "background-color:rgb(250,200,200)"
        return "color:%s"%c # it's just CSS, so you can do background as well.
    return df.style.applymap(color_how_good , subset=["k_vs_s_%"])


In [130]:
merged_df_with_meta.head()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code
0,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,2,2,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
1,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-13,1,0,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
2,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-14,1,1,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg
3,00000b54-600a-4de2-8700-fd9885252dca,PageView,https://www.moneysmart.sg/embed/98e61305602380...,2020-01-12,1,1,shop,/embed/98e61305602380971d9c5e68c4a75647,/embed,control,sg
4,00000b54-600a-4de2-8700-fd9885252dca,Reading,https://blog.moneysmart.sg/career/5-easy-side-...,2020-01-12,3,3,blog,/career/5-easy-side-businesses-you-can-run-whi...,/career,control,sg


## By Country


In [131]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "date"])

In [132]:
colour_grouped_table(g)

,country_code,date,s_count,k_count,k_vs_s_%
0,??,2020-01-10,507,26,-94.9
1,??,2020-01-11,345,14,-95.9
2,??,2020-01-12,362,29,-92
3,??,2020-01-13,669,42,-93.7
4,??,2020-01-14,648,46,-92.9
5,hk,2020-01-10,97635,96753,-0.9
6,hk,2020-01-11,86904,85923,-1.1
7,hk,2020-01-12,88044,87417,-0.7
8,hk,2020-01-13,92733,91840,-1
9,hk,2020-01-14,91012,90247,-0.8


## By Event Type

In [133]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "event_name"])
g.sort_values(["country_code", "event_name"], inplace=True)
colour_grouped_table(g)

,country_code,event_name,s_count,k_count,k_vs_s_%
4,??,LeadGeneration.ClickedCTA,1,1,0
14,??,PageView,2530,79,-96.9
15,??,Reading,0,74,999
23,??,UserEngagement.ButtonClick,0,3,999
36,hk,Display user feedback form,3,0,-100
38,hk,LeadGeneration.ClickConversion,501,529,5.6
39,hk,LeadGeneration.ClickedApplyButton,1712,1791,4.6
40,hk,LeadGeneration.ClickedCTA,1915,2018,5.4
42,hk,LeadGeneration.Conversion,4689,4595,-2
44,hk,LeadGeneration.FormSubmitted,993,1014,2.1


In [134]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.event_name=="LeadGeneration.ClickedApplyButton"], ["country_code", "slug_root", "event_name"])

In [135]:
g

,country_code,slug_root,event_name,s_count,k_count,k_vs_s_%
3,hk,/credit-cards,LeadGeneration.ClickedApplyButton,1712.0,1791.0,4.6
12,sg,/credit-cards,LeadGeneration.ClickedApplyButton,2806.0,2996.0,6.8
13,sg,/debt-consolidation-plan,LeadGeneration.ClickedApplyButton,115.0,124.0,7.8
14,sg,/personal-loan,LeadGeneration.ClickedApplyButton,871.0,914.0,4.9


## By Top Level Slug

In [136]:
g = group_by_and_show_count_difference(merged_df_with_meta[merged_df_with_meta.page_type!="blog"], ["country_code", "slug_root", "event_name"])
#g.sort_values(["country_code", "slug_root", "event_name"])

#filtering where s_count or k_count is >1000
pv = pd.pivot_table(g[(g.s_count>1000) | (g.k_count>1000)], index=["country_code", "slug_root"], values=["k_count","s_count","k_vs_s_%"], columns=["event_name"], fill_value="")

#colour_grouped_table(pv)
#TODO: not showing up the s_count and k_count :(
pv = pv.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [137]:
pv.to_excel("kinesis_vs_segment.xlsx")

In [138]:

def highlight_cols(cell):
    #use hex colours, or named ones to ensure excel compatibility on export
    if cell=="":
        return ""
    ci = min(100, int(abs(cell*10)))
    if abs(cell)<=2:
        return "color:green;"
    if cell <0:
        return "background-color:#%02x%02x%02x;" % (255,255-ci,255-ci)
    if cell>0:
        return "background-color:#%02x%02x%02x);" % (255-ci,255-ci,255)
    return "background-color:red;"



#pv[:20].style.background_gradient(cmap=cm, subset=pd.IndexSlice[:, 's-count'])
styled = pv.style.applymap(highlight_cols, subset=pv.columns.get_loc_level('k_vs_s_%', level=1)[0]) #special multi-index on column for colour
#can't get they styling to be happy with colors
#styled.to_excel("kinesis_vs_segment.xlsx", engine='openpyxl') #use special engine for formatting https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html

styled

In [139]:
pv.to_html("kinesis_vs_segment.html")

## By Type of Page

In [140]:
g = group_by_and_show_count_difference(merged_df_with_meta, ["country_code", "page_type"])
colour_grouped_table(g)

,country_code,page_type,s_count,k_count,k_vs_s_%
3,??,shop,2531,157,-93.8
5,hk,blog,284524,283311,-0.4
6,hk,iss,16674,16488,-1.1
7,hk,lps,5323,5387,1.2
8,hk,shop,149807,146994,-1.9
10,id,blog,0,4592,999
15,ph,blog,0,18389,999
20,sg,blog,1.19744e+06,1.20732e+06,0.8
21,sg,iss,31255,28560,-8.6
22,sg,lps,2274,2282,0.4


### Type of Page, Just Pageviews

In [158]:
g = group_by_and_show_count_difference(merged_df_with_meta[(merged_df_with_meta.event_name=="PageView") & (merged_df_with_meta.country_code.isin(["sg", "hk"]))], ["country_code", "page_type"])
colour_grouped_table(g)

,country_code,page_type,s_count,k_count,k_vs_s_%
5,hk,blog,91864,91450,-0.5
6,hk,iss,7722,7530,-2.5
7,hk,lps,4532,4531,-0
8,hk,shop,88968,87156,-2
20,sg,blog,427151,433719,1.5
21,sg,iss,12898,11610,-10
22,sg,lps,2097,2088,-0.4
23,sg,shop,355496,344372,-3.1
24,sg,unbounce,6083,6139,0.9


# Other Issues to Check For
* duplicates
* skipping "https" from the url (observed as a current issue)
* certain browsers having issues
* users creating a lot of duplicate events (and doing the above analysis using sum vs count)

# Play Area

In [163]:
d = merged_df_with_meta[merged_df_with_meta.page_type=="iss"].groupby(["slug", "page_type"]).sum()
d[d.s_count>0]
merged_df_with_meta[(merged_df_with_meta.page_type=="iss") & (merged_df_with_meta.page_url.str.contains("iss."))]

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code
779,0022082d-53bb-408c-80be-66d534cf584e,LeadGeneration.RedirectCompleted,https://iss.moneysmart.sg/credit-cards/standar...,2020-01-13,1,1,iss,/credit-cards/standard-chartered-unlimited-cas...,/credit-cards,control,sg
780,0022082d-53bb-408c-80be-66d534cf584e,PageView,https://iss.moneysmart.sg/credit-cards/standar...,2020-01-13,1,1,iss,/credit-cards/standard-chartered-unlimited-cas...,/credit-cards,control,sg
1184,0031f6a5-8883-4f0b-9213-6dc210a2a8e5,LeadGeneration.RedirectCompleted,https://iss.moneysmart.sg/credit-cards/krisfly...,2020-01-14,1,1,iss,/credit-cards/krisflyer-uob-credit-card/redirect,/credit-cards,control,sg
1185,0031f6a5-8883-4f0b-9213-6dc210a2a8e5,PageView,https://iss.moneysmart.sg/credit-cards/krisfly...,2020-01-14,1,1,iss,/credit-cards/krisflyer-uob-credit-card/redirect,/credit-cards,control,sg
1981,0051c38c-301d-4ba0-8364-b5ef9738225f,LeadGeneration.RedirectCompleted,https://iss.moneysmart.sg/credit-cards/standar...,2020-01-14,1,1,iss,/credit-cards/standard-chartered-unlimited-cas...,/credit-cards,control,sg
1984,0051c38c-301d-4ba0-8364-b5ef9738225f,PageView,https://iss.moneysmart.sg/credit-cards/standar...,2020-01-14,1,1,iss,/credit-cards/standard-chartered-unlimited-cas...,/credit-cards,control,sg
2612,006c5cc3-fc65-47b6-a25d-44d8ea8abc96,LeadGeneration.RedirectCompleted,https://iss.moneysmart.sg/credit-cards/revolut...,2020-01-14,1,1,iss,/credit-cards/revolut-premium-debit-card/redirect,/credit-cards,control,sg
2613,006c5cc3-fc65-47b6-a25d-44d8ea8abc96,LeadGeneration.RedirectCompleted,https://iss.moneysmart.sg/credit-cards/uob-prv...,2020-01-14,1,1,iss,/credit-cards/uob-prvi-miles-card/redirect,/credit-cards,control,sg
2614,006c5cc3-fc65-47b6-a25d-44d8ea8abc96,PageView,https://iss.moneysmart.sg/credit-cards/revolut...,2020-01-14,1,1,iss,/credit-cards/revolut-premium-debit-card/redirect,/credit-cards,control,sg
2615,006c5cc3-fc65-47b6-a25d-44d8ea8abc96,PageView,https://iss.moneysmart.sg/credit-cards/uob-prv...,2020-01-14,3,3,iss,/credit-cards/uob-prvi-miles-card/redirect,/credit-cards,control,sg


In [154]:
merged_df_with_meta[(merged_df_with_meta.slug_root=="/zh-hk") & (merged_df_with_meta.country_code=="hk") & (merged_df_with_meta.page_type!="blog")].head(40) #.groupby(["slug"]).sum()

,anonymous_id,event_name,page_url,date,s_count,k_count,page_type,slug,slug_root,ab_test,country_code
503,0013ff37-3670-495f-a554-99c5b53bc2db,PageView,https://www.moneysmart.hk/zh-hk,2020-01-14,1,0,shop,/zh-hk,/zh-hk,control,hk
1612,0040f831-6321-4c4f-8c18-805fb7992763,PageView,https://www.moneysmart.hk/zh-hk,2020-01-13,1,1,shop,/zh-hk,/zh-hk,control,hk
1613,0040f831-6321-4c4f-8c18-805fb7992763,PageView,https://www.moneysmart.hk/zh-hk,2020-01-14,1,1,shop,/zh-hk,/zh-hk,control,hk
1898,004d8a3b-ba5a-4890-be42-ad71a8db8ef8,PageView,https://www.moneysmart.hk/zh-hk,2020-01-10,2,2,shop,/zh-hk,/zh-hk,control,hk
2759,007090a7-fbc1-46b9-a2e1-44dd0aea9917,PageView,https://www.moneysmart.hk/zh-hk,2020-01-14,1,1,shop,/zh-hk,/zh-hk,control,hk
6127,0106c259-e8e8-4259-a93f-6cfc95988d26,PageView,https://www.moneysmart.hk/zh-hk,2020-01-13,1,0,shop,/zh-hk,/zh-hk,control,hk
10190,01b1c6b7-57bd-4523-8fc5-ddc74c5fe36a,PageView,https://www.moneysmart.hk/zh-hk,2020-01-14,3,1,shop,/zh-hk,/zh-hk,control,hk
13072,022684dd-b043-4911-a1cc-8cd86f9d4075,PageView,https://www.moneysmart.hk/zh-hk,2020-01-14,1,1,shop,/zh-hk,/zh-hk,control,hk
16914,02cd00b1-6830-4e73-972d-b662a3acd299,PageView,https://www.moneysmart.hk/zh-hk,2020-01-14,1,1,shop,/zh-hk,/zh-hk,control,hk
17020,02d1c2b1-5b86-4a8e-b5aa-660b60822c36,PageView,https://www.moneysmart.hk/zh-hk,2020-01-11,1,1,shop,/zh-hk,/zh-hk,control,hk


# Search Area

In [142]:
import ipywidgets as widgets


In [143]:
country_codes = merged_df_with_meta.country_code.unique().to_list()

In [144]:
country_codes

['sg', 'hk', '??', 'tw', 'id', 'ph']

In [145]:
top_level_slugs = list(merged_df_with_meta[merged_df_with_meta.page_type!="blog"].slug_root.unique())

In [146]:
top_level_slugs.sort()

In [164]:
event_types = list(merged_df_with_meta.event_name.unique())
event_types.sort()


['Display user feedback form',
 'EmailCapture',
 'LeadGeneration.ClickConversion',
 'LeadGeneration.ClickedApplyButton',
 'LeadGeneration.ClickedCTA',
 'LeadGeneration.ClickedGoToSite',
 'LeadGeneration.Conversion',
 'LeadGeneration.FormStepCompleted',
 'LeadGeneration.FormSubmitted',
 'LeadGeneration.PaymentCompleted',
 'LeadGeneration.RedirectCompleted',
 'LeadGeneration.ShowedMoreDetails',
 'LeadGeneration.ViewedMoreDetails',
 'LeadGeneration.VisitedSite',
 'PageView',
 'Reading',
 'UserAuth.ForgotPassword',
 'UserAuth.LogIn',
 'UserAuth.LogOut',
 'UserAuth.LoggedIn',
 'UserAuth.PhoneNumberVerified',
 'UserAuth.RegisteredAccount',
 'UserAuth.SignUp',
 'UserEngagement.ButtonClick',
 'UserEngagement.ClickedFilter',
 'UserEngagement.FilterSelection',
 'UserEngagement.QuestionAnswered',
 'UserEngagement.ShowMoreFilter',
 'UserEngagement.ShowMoreOptions',
 'UserEngagement.ShowedMoreDetails',
 'UserEngagement.SortedList',
 'UserEngagement.ViewedMoreDetails',
 'UserFeedback.FeedbackSubmitt

In [195]:
merged_df_with_meta.columns.to_list()

['anonymous_id',
 'event_name',
 'page_url',
 'date',
 's_count',
 'k_count',
 'page_type',
 'slug',
 'slug_root',
 'ab_test',
 'country_code']

In [199]:
def on_search_button_click(b):
    print("searching")
    print("erm...")
    df = merged_df_with_meta
    
    anonymous_id = anonymous_user_input.value.strip()
    search_anonymous_id = bool(anonymous_id)
    
    event_types = event_type_select.value
    search_event_types = len(event_types)>0
    
    slug_search_string = slug_search_input.value.strip()
    search_slug_by_string = bool(slug_search_string)
    
    country_codes = country_code_dropdown.value
    search_country_codes =  len(country_codes)!=0
        
    top_level_slugs = top_level_slug_select.value
    search_by_top_level_slugs = len(top_level_slugs)>0
    
    group_by_cols = group_by_select.value
    do_group_by = len(group_by_cols)>0
    
    search_mode = search_mode_dropdown.value
    print("search mode: %s"%search_mode)
    if not search_mode:
        print("No search mode set ")
        return
    
    if search_mode == "Events":
        print("Events search")
        d = df[(( not search_anonymous_id) | (df.anonymous_id==anonymous_id)) \
               & ( (not search_event_types) | (df.event_name.isin(event_types))) \
              & (( not search_slug_by_string) | (df.slug.str.contains(slug_search_string))) \
                & (( not search_country_codes) | (df.country_code.isin(country_codes))) \
               & (( not search_by_top_level_slugs) | (df.slug_root.isin(top_level_slugs))) \
               
               ]
        
        
        
        if do_group_by:
            d = d.groupby(group_by_cols).count()
        
        display(d)
    else:
        print("Search mode not supported yet")
    
    
    print("done searching")
    
def on_reset_button_click(b):
    print("I would be resetting")

In [200]:
def button_click_placeholder(b):
    print("just chilling")

anonymous_user_input = widgets.Text(description = "Anonymous_id")
country_code_dropdown = widgets.SelectMultiple(
    options= country_codes,
    value=["sg","hk"],
    # rows=10,
    description='Country',
    disabled=False
)
search_button = widgets.Button(description='Search', on_click=on_search_button_click)
reset_button = widgets.Button(description='Reset', on_click=on_reset_button_click)

search_button.on_click(on_search_button_click)

top_level_slug_select = widgets.SelectMultiple(options = top_level_slugs, description="slug")

slug_search_input = widgets.Text(description = "Slug like")

event_type_select = widgets.SelectMultiple(options=event_types, description="Event")


search_modes = ["Summary", "Summary Deduped", "Events",] # summary - > grouped with difference, events -> grouped by anon_id etc, 
search_mode_dropdown = widgets.Dropdown(description="Search Mode", options=search_modes, value=search_modes[0])

search_options = widgets.Box([anonymous_user_input, country_code_dropdown , top_level_slug_select, slug_search_input,  event_type_select])

group_by_select = widgets.SelectMultiple(description="Group By", options = merged_df_with_meta.columns.to_list())

search_bar = widgets.VBox([search_options, search_mode_dropdown, group_by_select, widgets.Box([search_button, reset_button])])
display(search_bar)



/bin/sh: True: command not found


In [ ]:
arrays = [np.hstack([['One']*2, ['Two']*2]) , ['A', 'B', 'C', 'D']]
columns = pd.MultiIndex.from_arrays(arrays)
data =  pd.DataFrame(np.random.randn(5, 4), columns=list('ABCD'))
data.columns = columns 

cm = sns.light_palette("green", as_cmap=True)


In [ ]:
data

In [ ]:
data.loc[pd.IndexSlice[:, pd.IndexSlice[:, 'A']]]

In [ ]:
data.columns

In [ ]:
data.style.background_gradient(cmap=cm, subset=data.columns.get_loc_level('A', level=1)[0])
